In [1]:
import ticktack
import ticktacktoe

In [3]:
MODULES = [ticktack, ticktacktoe]

In [5]:
for module in MODULES:
    cbm = module.load_presaved_model("Guttler14", 
        production_rate_units="atoms/cm^2/s")
    cbm.compile()
    cbm.equilibrate(target_C_14=707)